In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
%matplotlib inline

In [3]:
embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/1")

INFO:tensorflow:Using C:\Users\trive\AppData\Local\Temp\tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint b'C:\\Users\\trive\\AppData\\Local\\Temp\\tfhub_modules\\c6f5954ffa065cdb2f2e604e740e8838bf21a2d3\\variables\\variables' with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint b'C:\\Users\\trive\\AppData\\Local\\Temp\\tfhub_modules\\c6f5954ffa065cdb2f2e604e740e8838bf21a2d3\\variables\\variables' with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint b'C:\\Users\\trive\\AppData\\Local\\Temp\\tfhub_modules\\c6f5954ffa065cdb2f2e604e740e8838bf21a2d3\\variables\\variables' with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint b'C:\\Users\\trive\\AppData\\Local\\Temp\\tfhub_modules\\c6f5954ffa065cdb2f2e604e740e8838bf21a2d3\\variables\\variables

In [4]:
# Compute a representation for each message, showing various lengths supported.
'''word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))
    
    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        print("Message: {}".format(messages[i]))
        print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding[:3]))
        print("Embedding: [{}, ...]\n".format(message_embedding_snippet))'''

'word = "Elephant"\nsentence = "I am a sentence for which I would like to get its embedding."\nparagraph = (\n    "Universal Sentence Encoder embeddings also support short paragraphs. "\n    "There is no hard limit on how long the paragraph is. Roughly, the longer "\n    "the more \'diluted\' the embedding will be.")\nmessages = [word, sentence, paragraph]\n\n# Reduce logging output.\ntf.logging.set_verbosity(tf.logging.ERROR)\n\nwith tf.Session() as session:\n    session.run([tf.global_variables_initializer(), tf.tables_initializer()])\n    message_embeddings = session.run(embed(messages))\n    \n    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):\n        print("Message: {}".format(messages[i]))\n        print("Embedding size: {}".format(len(message_embedding)))\n        message_embedding_snippet = ", ".join(\n            (str(x) for x in message_embedding[:3]))\n        print("Embedding: [{}, ...]\n".format(message_embedding_snippet))'

In [5]:
train_data = pd.read_csv('train.csv')
train_data.drop(['id','qid1','qid2'],axis = 1,inplace = True)
#train_data['question1'][0],train_data['question2'][0]
len(train_data)
#train_data

404287

In [5]:
n = len(train_data)
#n = 10

def plot_similarity(labels, feature1,feature2, rotation):
    corr = np.inner(feature1,feature2)
    return corr
    #sns.set(font_scale=1.2)
    #g = sns.heatmap(
        #corr,
        #xticklabels=labels,
        #yticklabels=labels,
        #vmin=0,
        #vmax=1,
        #cmap="YlOrRd")
    #g.set_xticklabels(labels, rotation=rotation)
    #g.set_title("Semantic Textual Similarity")


def run_and_plot(session_, input_tensor_, messages_, encoding_tensor):
    message_embeddings_ = session_.run(encoding_tensor, feed_dict={input_tensor_: messages})
    #print(message_embeddings_[0])
    corr_list = []
    for i in range(0,2*n,2):
        corr_list.append(plot_similarity(messages_, message_embeddings_[i],message_embeddings_[i+1], 90))
    return (corr_list)

def message_embeddings(session_, input_tensor_, messages_, encoding_tensor):
    message_embeddings_ = session_.run(encoding_tensor, feed_dict={input_tensor_: messages})
    #print(message_embeddings_[0])
    embed = []
    for i in range(0,2*n,2):
        embed.append(np.concatenate([message_embeddings_[i],message_embeddings_[i+1]]))
    return (embed)

In [ ]:
import time 

start = time.time()

similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))

similarity_message_encodings = embed(similarity_input_placeholder)

messages = []
for i in range(n):
    messages.append(train_data['question1'][i])
    messages.append(train_data['question2'][i])
    
    
with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    feature = message_embeddings(session, similarity_input_placeholder, messages,
                                  similarity_message_encodings)
    #print(feature)
    #print(feature2)
    #print(corr_list)
    
print(time.time() - start)

#plt.figure(figsize=(10,6))
#plt.xlabel('Inner Product')
#plt.ylabel('Semantic Similarity label (0 or 1)')
#plt.scatter(corr_list,train_data['is_duplicate'][:n],c = train_data['is_duplicate'][:n])
#plt.gray()
#plt.legend()

In [7]:
check = pd.DataFrame(feature)
check['label'] = train_data['is_duplicate'][:n]
check.to_csv('Embeddings.csv')
check.shape

(120000, 1025)

In [6]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
# fix random seed for reproducibility
np.random.seed(7)
dataset = np.loadtxt("Embeddings_full.csv",delimiter=',',skiprows=1)
X = dataset[:,1:1025]

Y = dataset[:,1025]


Using TensorFlow backend.


In [7]:
X_new = np.zeros((len(X),512))

for i in range(len(X)):
    for j in range(512):
        X_new[i][j] = X[i][j] - X[i][512+j]

In [8]:
X1 = np.concatenate((X, X_new), axis=1)

In [9]:
# create training and testing vars
validation_size = 40000
X_train, X_test, y_train, y_test = train_test_split(X1, Y, test_size = validation_size)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)
#Y = np.array(train_data['is_duplicate'][:n])
#Y

(364287, 1536) (364287,)
(40000, 1536) (40000,)


In [11]:
import time
start = time.time()
# create model
model = Sequential()
model.add(Dense(12, input_dim=1536, activation='relu'))
model.add(Dense(30, activation='relu')) #tanh Bruteforce for activation function
model.add(Dense(30, activation='relu')) #tanh Bruteforce for activation function
model.add(Dense(30, activation='relu')) #tanh Bruteforce for activation function
model.add(Dense(1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
result = model.fit(X_train, y_train, epochs=50, batch_size=10,validation_data = (X_test, y_test))

# evaluate the model
scores = model.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

print("Test-Accuracy:", np.mean(result.history["val_acc"]))

# calculate predictions
#predictions = model.predict(X_test)
# round predictions
#rounded = [round(x[0]) for x in predictions]
#print(rounded)

print(time.time() - start)

Train on 364287 samples, validate on 40000 samples
Epoch 1/50
364287/364287 [==============================] - 468s 1ms/step - loss: 0.4580 - acc: 0.7704 - val_loss: 0.4383 - val_acc: 0.7834
Epoch 2/50
364287/364287 [==============================] - 172s 471us/step - loss: 0.4270 - acc: 0.7912 - val_loss: 0.4264 - val_acc: 0.7925
Epoch 3/50
364287/364287 [==============================] - 103s 284us/step - loss: 0.4177 - acc: 0.7973 - val_loss: 0.4251 - val_acc: 0.7931
Epoch 4/50
364287/364287 [==============================] - 66s 180us/step - loss: 0.4126 - acc: 0.8002 - val_loss: 0.4195 - val_acc: 0.7950
Epoch 5/50
364287/364287 [==============================] - 59s 161us/step - loss: 0.4084 - acc: 0.8028 - val_loss: 0.4225 - val_acc: 0.7959
Epoch 6/50
364287/364287 [==============================] - 59s 161us/step - loss: 0.4057 - acc: 0.8045 - val_loss: 0.4177 - val_acc: 0.7957
Epoch 7/50
364287/364287 [==============================] - 60s 165us/step - loss: 0.4030 - acc: 0.806